# Mass-Spring-Damper System

As the first example for frequency domain modeling methods, we consider a classical mechanical mass-spring-damper system with a single chain of masses connected by springs and independent dampers on every mass. This system with $k$ masses can be described by second-order generalized differential equations of the form

$$
\tag{4.1}
\begin{aligned}
    m_{1} \ddot{p}_{1}(t) + \gamma_{1} \dot{p}_{1}(t) + \kappa_{1}(p_{1}(t) - p_{2}(t))
    &= u(t),
    \\
    m_{i} \ddot{p}_{i}(t) + \gamma_{i} \dot{p}_{i}(t) + \kappa_{i} (p_{i}(t) - p_{i + 1}(t)) - \kappa_{i-1} (p_{i - 1}(t) - p_{i}(t))
    &= 0 \quad \text{for}~~i = 1, \ldots, k - 1,
    \\
    m_{k} \ddot{p}_{k}(t) + \gamma_{k} \dot{p}_{k}(t) + \kappa_{p} p_{k}(t) - \kappa_{k-1} (p_{k - 1}(t) - p_{k}(t))
    &= 0,
\end{aligned}
$$

with mass, damping, and spring stiffness parameters $m_{i}, \gamma_{i}, \kappa_{i} > 0,$ for $i = 1, \ldots, k$.
The external forcing $u(t)$ acts only on the first mass.
The quantity of interest is the velocity of the first mass in the chain,

$$
    y(t) = \dot{p}_{1}(t).
$$

Using the extended state

$$
    \mathbf{q}(t) = \begin{bmatrix}
        p_1(t) \\ \vdots \\ p_k(t)
        \\
        \dot{p}_1(t) \\ \vdots \\ \dot{p}_k(t)
    \end{bmatrix} \in \mathbb{R}^{2k},
$$

the dynamical system $(4.1)$ can be written in the classical linear form

$$
\tag{4.2}
\begin{aligned}
    \mathbf{E} \dot{\mathbf{q}}(t)
    &= \mathbf{A} \mathbf{q}(t) + \mathbf{b} u(t),
    \\
    y(t)
    &= \mathbf{c}^\mathsf{T} \mathbf{q}(t)
\end{aligned}
$$

where $\mathbf{E},\mathbf{A}\in\mathbb{R}^{n\times n}$ and $\mathbf{b},\mathbf{c}\in\mathbb{R}^{n}$ with $n = 2k$.
Because the input $u(t)$ and the output $y(t)$ are both one-dimensional, this is a *single-input single-output* (SISO) system.

The *transfer function* corresponding to $(4.2)$ is given by

$$
\tag{4.3}
\begin{aligned}
    G(s) = \mathbf{c}^\mathsf{T} (s \mathbf{E} - \mathbf{A})^{-1} \mathbf{b},
\end{aligned}
$$

which satisfies $Y(s) = G(s)U(s)$ where $U(s)$ and $Y(s)$ are the Laplace transforms of $u(t)$ and $y(t),$ respectively.

For given measurements of the transfer function $G(s),$ our objective is to find potentially low-dimensional matrices $\widehat{\mathbf{A}},$ $\widehat{\mathbf{b}},$ $\widehat{\mathbf{c}},$ and $\widehat{\mathbf{E}}$ so that the corresponding transfer function

$$
\widehat{G}(s)
= \widehat{\mathbf{c}}^\mathsf{T} (s \widehat{\mathbf{E}} - \widehat{\mathbf{A}})^{-1} \widehat{\mathbf{b}}
$$

approximates the given data well.
Since the matrix system structure is retained, the matrices $\widehat{\mathbf{A}},$ $\widehat{\mathbf{b}},$ $\widehat{\mathbf{c}},$ and $\widehat{\mathbf{E}}$ can be viewed as taking the role of the system matrices $\mathbf{A},$ $\mathbf{b},$ $\mathbf{c},$ and $\mathbf{E}$, respectively, in the time-domain description of the system.

**Note**: This data fitting problem is a *rational approximation* problem, so named because the transfer function to be constructed is a rational function.

First, we import some standard Python packages for our computations and the visualization.

In [ ]:
import numpy as np
import scipy.linalg as la
import scipy.sparse as sparse
import matplotlib.pyplot as plt

System construction and some visualization routines are implemented in the file [utils.py](./utils.py).

In [ ]:
import utils

utils.configure_matplotlib(latex_is_installed = True)

The file also defines two important classes that we will use:

- The `SISO` class represents general linear systems of the form $(4.2)$.
- The `MassSpringDamper` class constructs the particular SISO system $(4.1)$.

Both classes have `A`, `b`, `c`, and `E` attributes representing the system matrices, as well as a simple `transfer_function()` method for evaluating $(4.3)$ which boils down to the following line.

```python
    response = c @ scipy.linalg.solve(s * E - A, b)
```

To begin, we generate a mass-spring-damper system with just $2$ masses (hence $4$ degrees of freedom) and some standard parameters. Later in this demonstration, we will consider the high-dimensional case in which dimensionality reduction plays a role.

In [ ]:
truth_model = utils.MassSpringDamper(
    num_masses = 2, mass = 1.0, damping = 0.1, stiffness = 10.0
)

print(
    "State matrix A:",
    truth_model.A.toarray(),
    sep="\n",
)

print(truth_model.c)

Let $\mathfrak{j} = \sqrt{-1}$ be the imaginary unit, in Python given by `1j`. The next block plots the magnitude of transfer function values, $|G(\mathrm{j}\omega)|,$ for $500$ logarithmically spaced frequencies $\omega$ in the interval $[10^{-2}, 10^{2}]$ rad/s. We call the transfer function values the *frequency response* of the system.

In [ ]:
frequencies = np.logspace(-2, 2, 500)
responses   = truth_model.transfer_function(1j * frequencies)

utils.plot_response(frequencies, responses)
plt.show()

This represents the function that we are aiming to approximate via frequency domain data using the Loewner framework.

## Step 1: Generate Training Data

We will learn a dynamical system from a limited number of transfer function samples. Due to the amount of degrees of freedom when designing systems with the Loewner framework, we need an **even** number of data samples.

In [ ]:
# Number of data samples generated (this must be an even integer).
num_samples = 8

# Sample the transfer function.
training_frequencies = np.logspace(-2, 2, num_samples)
training_responses   = truth_model.transfer_function(1j * training_frequencies)

# Plot the data samples over the actual transfer function.
ax = utils.plot_response(
    frequencies,
    responses,
    label = "Transfer function"
)
ax.loglog(
    training_frequencies,
    np.abs(training_responses),  # Responses are complex, so plot magnitude.
    "ks",
    label = "Samples"
)
ax.legend(loc = "upper left")
plt.show()

**Note**: In this example, exactly $8$ data samples are needed to construct a highly accuracy model, since there are $n = 4$ degrees of freedom and the samples are split into two even groups of $4$ (detailed explanation follows below). Try experimenting with fewer samples.

## Step 2: Construct Loewner matrices

The Loewner framework builds a system whose transfer function, by construction, interpolates the given data.

### Split the Data into Left and Right Sets

Let $\omega_1, \omega_2, \ldots$ denote the training frequencies and define the interpolation points $\mu_i = \mathfrak{j}\omega_i$.
The goal is to construct $\widehat{G}:\mathbb{C}\to\mathbb{C}$ so that $\widehat{G}(\mu_i) = G(\mu_i)$ for all $i$.

First, we split the data pairs $\{(\mu_i, G(\mu_i))\}_i$ into two subsets of equal size of the form $(\mu_{\operatorname{\ell}}, g_{\operatorname{\ell}})$ and $(\mu_{\operatorname{r}}, g_{\operatorname{r}})$. These are referred to as left and right datasets. While the distribution of data does not play any theoretical role, we select alternating data points from the full set of samples.

In [ ]:
# Left data pairs.
w_l  = training_frequencies[0::2]
g_l  = training_responses[0::2]
mu_l = 1j * w_l

# Right data pairs.
w_r  = training_frequencies[1::2]
g_r  = training_responses[1::2]
mu_r = 1j * w_r

# Plot the data samples over the actual transfer function.
ax = utils.plot_response(frequencies, responses, label = "Transfer function")
utils.plot_samples(w_l, g_l, w_r, g_r, ax = ax)
ax.legend(loc = "upper left")
plt.show()

### Construct Loewner Matrices and the Corresponding System

The Loewner ROM is defined in terms of the matrices

$$
\begin{aligned}
    \mathbb{L}
    &= \begin{bmatrix}
        \frac{g_{\operatorname{\ell}, 1} - g_{\operatorname{r}, 1}}{\mu_{\operatorname{\ell}, 1} - \mu_{\operatorname{r}, 1}} & \frac{g_{\operatorname{\ell}, 1} - g_{\operatorname{r}, 2}}{\mu_{\operatorname{\ell}, 1} - \mu_{\operatorname{r}, 2}} & \cdots
        \\[.25cm]
        \frac{g_{\operatorname{\ell}, 2} - g_{\operatorname{r}, 1}}{\mu_{\operatorname{\ell}, 2} - \mu_{\operatorname{r}, 1}} & \frac{g_{\operatorname{\ell}, 2} - g_{\operatorname{r}, 2}}{\mu_{\operatorname{\ell}, 2} - \mu_{\operatorname{r}, 2}} &  \cdots
        \\[.25cm]
        \vdots & \vdots & \ddots
    \end{bmatrix} \in \mathbb{C}^{kp \times km},
    &
    \mathbb{L}_{\operatorname{s}} &= \begin{bmatrix}
        \frac{\mu_{\operatorname{\ell}, 1}g_{\operatorname{\ell}, 1} - \mu_{\operatorname{r}, 1}g_{\operatorname{r}, 1}}{\mu_{\operatorname{\ell}, 1} - \mu_{\operatorname{r}, 1}} & \frac{\mu_{\operatorname{\ell}, 1}g_{\operatorname{\ell}, 1} - \mu_{\operatorname{r}, 2}g_{\operatorname{r}, 2}}{\mu_{\operatorname{\ell}, 1} - \mu_{\operatorname{r}, 2}} & \cdots
        \\[.25cm]
        \frac{\mu_{\operatorname{\ell}, 2}g_{\operatorname{\ell}, 2} - \mu_{\operatorname{r}, 1}g_{\operatorname{r}, 1}}{\mu_{\operatorname{\ell}, 2} - \mu_{\operatorname{r}, 1}} & \frac{\mu_{\operatorname{\ell}, 2} g_{\operatorname{\ell}, 2} - \mu_{\operatorname{r}, 2} g_{\operatorname{r}, 2}}{\mu_{\operatorname{\ell}, 2} - \mu_{\operatorname{r}, 2}} &  \cdots
        \\[.25cm]
        \vdots & \vdots & \ddots
    \end{bmatrix} \in \mathbb{C}^{kp \times km},
    \\
    \mathbf{B}_{\mathbb{L}}
    &= \begin{bmatrix}
        g_{\operatorname{\ell}, 1} \\
        g_{\operatorname{\ell}, 2} \\
        \vdots
    \end{bmatrix} \in \mathbb{C}^{kp \times m},
    &
    \mathbf{C}_{\mathbb{L}}
    &= \left[\begin{array}{ccc}
        g_{\operatorname{r}, 1} & g_{\operatorname{r}, 2} & \cdots
    \end{array}\right] \in \mathbb{C}^{p \times km},
\end{aligned}
$$

using the separated data sets $(\mu_{\operatorname{\ell}}, g_{\operatorname{\ell}})$ and $(\mu_{\operatorname{r}}, g_{\operatorname{r}})$ of lengths $k$.
Here, $m$ and $p$ are the numbers of inputs and outputs to the system, respectively; in this single-input single-output problem, $m = p = 1$.
In general, we call $\mathbb{L}$ a *Loewner matrix* and $\mathbb{L}_{\operatorname{s}}$ a *shifted Loewner matrix*.

The system whose transfer function interpolates the data is then defined by the matrices

$$
\begin{aligned}
    \widehat{\mathbf{A}}
    &= -\mathbb{L}_{\operatorname{s}}
    &
    \widehat{\mathbf{b}}
    &= \mathbf{B}_{\mathbb{L}},
    &
    \widehat{\mathbf{c}}
    &= \mathbf{C}_{\mathbb{L}},
    &
    \widehat{\mathbf{E}} &= -\mathbb{L}.
\end{aligned}
$$

**Note**: This is system identification, not model reduction, as there is no dimensionality reduction (yet).

In [ ]:
def loewner_matrices(mu_l, g_l, mu_r, g_r):
    """Construct the Loewner matrices L, Ls, BL, and CL."""
    BL = g_l
    CL = g_r

    k  = len(mu_l)
    L  = np.zeros((k, k), dtype = complex)
    Ls = np.zeros((k, k), dtype = complex)

    # Explicit entrywise construction.
    # for i in range(k):
    #     for j in range(k):
    #         L[i, j] = (g_l[i] - g_r[j]) / (mu_l[i] - mu_r[j])
    #         Ls[i, j] = (mu_l[i] * g_l[i] - mu_r[j] * g_r[j]) / (mu_l[i] - mu_r[j])

    # Array broadcasting construction.
    g_l  = g_l.reshape((k, 1))  # Make these into column vectors.
    mu_l = mu_l.reshape((k, 1))

    L  = (g_l - g_r) / (mu_l - mu_r)
    Ls = ((mu_l * g_l) - (mu_r * g_r)) / (mu_l - mu_r)

    return L, Ls, BL, CL

In [ ]:
L, Ls, BL, CL = loewner_matrices(mu_l, g_l, mu_r, g_r)

loewner_model = utils.SISO(A = -Ls, b = BL, c = CL, E = -L)

## Step 3: Evaluate the Learned Model

Model accuracy in this setting is evaluated by checking the transfer function.

### Sanity Check: Compare to Training Data

First, we compare the frequency response of the learned model at the training frequencies.
These should match up to machine precision because this method is interpolatory.

In [ ]:
model_training_reponses = loewner_model.transfer_function(
    1j * training_frequencies
)

for w, g_true, g_model in zip(
    training_frequencies, training_responses, model_training_reponses
):
    print(
        f"Frequency: {w:.4e}",
        f"Truth: {g_true.real: >10.3e} + {g_true.imag: >10.3e}j",
        f"Loewner: {g_model.real: >10.3e} + {g_model.imag: >10.3e}j",
        sep="    ",
    )

### Predict at Non-Training Frequencies

Next, we can also have a look at the pointwise relative error for general frequencies,

$$
\operatorname{relerr}(\omega) = \frac{\lvert G(\mathfrak{j} \omega) - \widehat{G}(\mathfrak{j} \omega) \rvert}{\lvert G(\mathfrak{j} \omega) \rvert}.
$$

The following code block compares the transfer function of the learned model to that of the truth model.

In [ ]:
# Evaluate the transfer function of the learned model at many frequencies.
model_responses = loewner_model.transfer_function(1j * frequencies)

# Calculate the relative error based on the true frequency reponses.
model_relative_errors = np.abs(model_responses - responses) / np.abs(responses)

# Visualize the results.
utils.plot_comparison(
    frequencies,
    responses,
    model_responses,
    model_relative_errors,
    w_l,
    g_l,
    w_r,
    g_r,
)
plt.show()

**Note**: If the true $\mathbf{A},$ $\mathbf{b},$ $\mathbf{c},$ and $\mathbf{E}$ are available, a SISO system can be constructed that produces the (absolute) error by

$$
\begin{aligned}
    G(s) - \widehat{G}(s)
    = \begin{bmatrix}
        \mathbf{c} \\ -\widehat{\mathbf{c}}
    \end{bmatrix}^\mathsf{T}
    \left(s \begin{bmatrix}
        \mathbf{E} & 0 \\ 0 & \widehat{\mathbf{E}}
    \end{bmatrix} - \begin{bmatrix}
        \mathbf{A} & 0 \\ 0 & \widehat{\mathbf{A}}
    \end{bmatrix} \right)^{-1}
    \begin{bmatrix} \mathbf{b} \\ \widehat{\mathbf{b}} \end{bmatrix}.
\end{aligned}
$$

In [ ]:
# Construct the error system.
error_model = utils.SISO(
    A = sparse.block_diag((truth_model.A, loewner_model.A)),  # [A, 0; 0, Ahat]
    b = np.concatenate((truth_model.b, loewner_model.b)),  # [b; bhat]
    c = np.concatenate((truth_model.c, -loewner_model.c)),  # [c; -chat]
    E = sparse.block_diag((truth_model.E, loewner_model.E)),  # [E, 0; 0, Ehat]
)

# Evaluate the error system transfer function, which gives the absolute error.
error_model_response = error_model.transfer_function(1j * frequencies)

# Compare the (relative) error to the previous computation.
np.allclose(error_model_response / np.abs(responses), model_relative_errors)

**Important**: Even though the response of the learned system matches the true system response up to machine precision, the learned matrices $\widehat{\mathbf{A}},$ $\widehat{\mathbf{b}},$ $\widehat{\mathbf{c}},$ and $\widehat{\mathbf{E}}$ are not necessarily equal to the original $\mathbf{A},$ $\mathbf{b},$ $\mathbf{c},$ and $\mathbf{E}$. However, they are equivalent and can be related by a state-space transformation, meaning there exist invertible matrices $\mathbf{W},\mathbf{T}\in\mathbb{C}^{n\times n}$ such that

$$
    \mathbf{A} = \mathbf{W} \widehat{\mathbf{A}} \mathbf{T},
    \quad
    \mathbf{b} = \mathbf{W} \widehat{\mathbf{b}},
    \quad
    \mathbf{c} = \widehat{\mathbf{c}} \mathbf{T},
    \quad
    \mathbf{E} = \mathbf{W} \widehat{\mathbf{E}} \mathbf{T}.
$$

In [ ]:
print(
    f"Shape of   truth model A: {truth_model.A.shape}",
    f"Shape of learned model A: {loewner_model.A.shape}",
    sep="\n",
)

# The state matrices are not the same.
# (This will raise an error if num_samples != 8.)
print(
    "\nA matrices are the same:",
    np.allclose(truth_model.A.toarray(), loewner_model.A),
)

## Extension 1: More Data

The Loewner framework constructs an interpolating system whose dimension is half the number of data samples, which is why using $8$ samples results in interpolation when the truth model has dimension $n = 2k = 4$. When more data samples are available, the Loewner model is overfitting and an adjustment is needed. First, observe that the rank of the Loewner pencil matrices

$$
    r
    = \operatorname{rank}\left(\begin{bmatrix}
        \mathbb{L}_{\operatorname{s}} \\ \mathbb{L}
    \end{bmatrix}\right)
    = \operatorname{rank}\left(\begin{bmatrix}
        \mathbb{L}_{\operatorname{s}} & \mathbb{L}
    \end{bmatrix}\right)
$$

corresponds to the system rank in the sense that $r \leq n$.

In [ ]:
s1 = la.svdvals(np.vstack((L, Ls)))  # [L; Ls]
s2 = la.svdvals(np.hstack((L, Ls)))  # [L, Ls]

utils.plot_singular_values(s1, s2)
plt.show()

These pencils have rank $r = 4,$ which is exactly the dimension of the original system.
If we have $n_s = 12 > 2n$ data samples, the Loewner model constructs a linear system of dimension $r = n_s / 2 = 6$.

In [ ]:
num_samples = 12

# Sample the transfer function.
training_frequencies = np.logspace(-2, 2, num_samples)
training_responses   = truth_model.transfer_function(1j * training_frequencies)

# Left data pairs.
w_l  = training_frequencies[0::2]
g_l  = training_responses[0::2]
mu_l = 1j * w_l

# Right data pairs.
w_r  = training_frequencies[1::2]
g_r  = training_responses[1::2]
mu_r = 1j * w_r

# Construct Loewner matrices.
L, Ls, BL, CL = loewner_matrices(mu_l, g_l, mu_r, g_r)

# Compute the singular values of the pencil matrices.
s1 = la.svdvals(np.vstack((L, Ls)))  # [L; Ls]
s2 = la.svdvals(np.hstack((L, Ls)))  # [L, Ls]

utils.plot_singular_values(s1, s2)
plt.show()

The rank of the Loewner pencils is still $r = 4,$ with larger singular values being numerically zero. Using these matrices would result in a singular system, i.e., $s\widehat{\mathbf{E}} - \widehat{\mathbf{A}}$ is not invertible for any $s \in \mathbb{C}$. To recover an $r = 4$ dimensional system, we use the singular value decompositions of the two concatenated matrices above,

$$
    \boldsymbol{\Phi}_1\boldsymbol{\Sigma}_1\boldsymbol{\Psi}_1^\mathsf{H}
    = \begin{bmatrix} \mathbb{L}_{\operatorname{s}} \\ \mathbb{L} \end{bmatrix}
    \quad\text{and}\quad
    \boldsymbol{\Phi}_2\boldsymbol{\Sigma}_2\boldsymbol{\Psi}_2^\mathsf{H}
    = \begin{bmatrix} \mathbb{L}_{\operatorname{s}} & \mathbb{L} \end{bmatrix}.
$$

The orthogonal bases of degree $r$ resulting from these decompositions can be used to truncate the Loewner matrices and construct the appropriate system:

$$
\begin{aligned}
    \widehat{\mathbf{A}}
    &= - \boldsymbol{\Phi}_{2,r}^\mathsf{H} \mathbb{L}_{s} \boldsymbol{\Psi}_{1,r},
    &
    \widehat{\mathbf{b}}
    &= \boldsymbol{\Phi}_{2,r}^\mathsf{H} \mathbf{B}_{\mathbb{L}}
    &
    \widehat{\mathbf{c}}
    &= \mathbf{C}_{\mathbb{L}}\boldsymbol{\Psi}_{1,r},
    &
    \widehat{\mathbf{E}}
    &= - \boldsymbol{\Phi}_{2,r}^\mathsf{H} \mathbb{L} \boldsymbol{\Psi}_{1,r},
\end{aligned}
$$

where $\boldsymbol{\Phi}_{2,r}$ comprises the first $r$ columns of $\boldsymbol{\Phi}_2$ and similar for $\boldsymbol{\Psi}_{1,r}$.

In [ ]:
# Compute the SVDs of the Loewner pencil.
Phi_1, s1, Psi_1T = la.svd(np.vstack((L, Ls)))
Phi_2, s2, Psi_2T = la.svd(np.hstack((L, Ls)))

# Determine the rank from the singular values.
r = np.count_nonzero(s1 > s1[0] * 1.0e-12)
print(f"Numerical rank based on SVD: {r:d}")

# Construct the truncated system.
Ar = -Phi_2[:, :r].conj().T @ (Ls @ Psi_1T[:r, :].conj().T)
br = Phi_2[:, :r].conj().T @ g_l
cr = g_r @ Psi_1T[:r, :].conj().T
Er = -Phi_2[:, :r].conj().T @ (L @ Psi_1T[:r, :].conj().T)

loewner_model2 = utils.SISO(A = Ar, b = br, c = cr, E = Er)

We correctly determined the rank to be $r = 4$ and constructed the corresponding Loewner system. Let's quickly check the results as before:

In [ ]:
# Evaluate the transfer function of the learned model at many frequencies.
model_responses2 = loewner_model2.transfer_function(1j * frequencies)

# Calculate the relative error based on the true frequency reponses.
model_relative_errors2 = np.abs(model_responses2 - responses) / np.abs(
    responses
)

# Visualize the results.
utils.plot_comparison(
    frequencies,
    responses,
    model_responses2,
    model_relative_errors2,
    w_l,
    g_l,
    w_r,
    g_r,
)
plt.show()

**Important**: This truncation approach can be used to construct reduced-order models with $r < n.$ We only need to determine a suitable $r,$ which is either chosen via a relative tolerance for the singular values of the Loewner pencil or via a maximum order set by the user. This is demonstrated at the end of this notebook. Note that if we truncate before the true pencil rank, i.e., choose $r$ with

$$
    r
    < \operatorname{rank}\left(\begin{bmatrix}
        \mathbb{L}_{\operatorname{s}} \\ \mathbb{L}
    \end{bmatrix}\right)
    = \operatorname{rank}\left(\begin{bmatrix}
        \mathbb{L}_{\operatorname{s}} & \mathbb{L}
    \end{bmatrix}\right),
$$

then the interpolation property of the Loewner framework is lost.
Empirically, it has been observed that by using the singular value decomposition for the truncation process, the resulting system fits the data in an *approximate least-squares* sense instead of interpolating.

## Extension 2: Realification

The learned matrices $\widehat{\mathbf{A}},$ $\widehat{\mathbf{b}},$ $\widehat{\mathbf{c}},$ and $\widehat{\mathbf{E}}$ constructed so far are complex-valued, even though the original system matrices are all real-valued.

In [ ]:
print(
    "True state matrix A:",
    truth_model.A.toarray(),
    "\nLoewner state matrix A:",
    loewner_model.A,
    sep="\n",
)

In many applications, we want to preserve the realness of the original system by enforcing realness of the learned matrices for any given data. To do this, we need the important observation that for a transfer function of a first-order system with real matrices, it holds that

$$
\overline{G(s)} = G(\overline{s}),
$$

which means that the transfer function values of complex conjugate evaluation points are also complex conjugates. The following code verifies this with our mass-spring-damper system example on four sets of complex conjugate points and the corresponding transfer function values.

In [ ]:
s = 1j * np.logspace(-2, 2, num = 4)

g1 = truth_model.transfer_function(s).conj()
g2 = truth_model.transfer_function(s.conj())

print(
    f"conj(G(s)):\n{g1}",
    f"G(conj(s)):\n{g2}",
    f"Numerically equal: {np.allclose(g1, g2)}",
    sep="\n\n",
)

We need this type of data to construct models with real matrices. As we typically do not have evaluations of transfer functions in the lower complex half-plane, we can artificially create the complex conjugate data from already given data. Once we have this, we need to split and reorder the data so that for the left and right data we have the complex conjugate data pairs in the same data set and interchanging, i.e.,

$$
\begin{aligned}
  \mu_{\operatorname{\ell}} & = \{ \mu_{\operatorname{\ell}, 1}, \overline{\mu_{\operatorname{\ell}, 1}}, \mu_{\operatorname{\ell}, 2}, \overline{\mu_{\operatorname{\ell}, 2}}, \ldots  \}, &
  g_{\operatorname{\ell}} & = \{ g_{\operatorname{\ell}, 1}, \overline{g_{\operatorname{\ell}, 1}}, g_{\operatorname{\ell}, 2}, \overline{g_{\operatorname{\ell}, 2}}, \ldots \}, \\
  \mu_{\operatorname{r}} & = \{ \mu_{\operatorname{r}, 1}, \overline{u_{\operatorname{r}, 1}}, u_{\operatorname{r}, 2}, \overline{u_{\operatorname{r}, 2}}, \ldots  \}, &
  g_{\operatorname{r}} & = \{ g_{\operatorname{r}, 1}, \overline{g_{\operatorname{r}, 1}}, g_{\operatorname{r}, 2}, \overline{g_{\operatorname{r}, 2}}, \ldots \}.
\end{aligned}
$$

In [ ]:
def insert_conjugates(arr):
    """From an array [x1, x2, ...], form [x1, conj(x1), x2, conj(x2), ...]"""
    new_arr       = np.empty(len(arr) * 2, dtype = complex)
    new_arr[::2]  = arr
    new_arr[1::2] = arr.conj()
    return new_arr

When we build the Loewner system with this data, there exists a state-space transformation that results in real system matrices. This transformation is given by

$$
    \mathbf{J}
    = \mathbf{I}_{k} \otimes \frac{1}{\sqrt{2}}
    \left[\begin{array}{rr}
        1 & \mathrm{j} \\ 1 & -\mathrm{j}
    \end{array}\right],
$$

and then applied so that

$$
    \widehat{\mathbf{A}}
    = -\mathbf{J}^\mathsf{H}\mathbb{L}_{\operatorname{s}} \mathbf{J},
    \quad
    \widehat{\mathbf{b}}
    = \mathbf{J}^\mathsf{H} g_{\operatorname{\ell}},
    \quad
    \widehat{\mathbf{c}}
    = g_{\operatorname{r}} \mathbf{J},
    \quad
    \widehat{\mathbf{E}} = -\mathbf{J}^\mathsf{H} \mathbb{L} \mathbf{J}.
$$

Let's do exactly that for our mass-spring damper example. We have already generated the necessary data above.

In [ ]:
# Generate only 4 samples (since conjugate data doubles the sample size).
num_samples = 4

# Sample the transfer function.
training_frequencies = np.logspace(-2, 2, num_samples)
training_responses   = truth_model.transfer_function(1j * training_frequencies)

# Split into left and right data sets AND add in the conjugates.
mu_l = insert_conjugates(1j * training_frequencies[::2])
mu_r = insert_conjugates(1j * training_frequencies[1::2])
g_l  = insert_conjugates(training_responses[::2])
g_r  = insert_conjugates(training_responses[1::2])

# Construct the transformation matrix J.
J = sparse.kron(
    sparse.eye(num_samples // 2),
    sparse.coo_matrix((1 / np.sqrt(2)) * np.array([[1, 1j], [1, -1j]])),
)

# Construct Loewner matrices.
L, Ls, BL, CL = loewner_matrices(mu_l, g_l, mu_r, g_r)

# Transform the Loewner system to real coordinates.
Ar = -J.conj().T @ (Ls @ J)
br = J.conj().T @ g_l
cr = g_r @ J
Er = -J.conj().T @ (L @ J)

# Check that the system matrices are real.
for label, arr in zip("AbcE", [Ar, br, cr, Er]):
    print(
        f"{arr.shape} matrix '{label}' has all real entries:",
        np.allclose(arr.real, arr),
    )

As before, we can check that our the resulting system is actually correct, by computing its transfer function and printing the error.

In [ ]:
loewner_model3 = utils.SISO(A = Ar, b = br, c = cr, E = Er)

# Evaluate the transfer function of the learned model at many frequencies.
model_responses3 = loewner_model3.transfer_function(1j * frequencies)

# Calculate the relative error based on the true frequency reponses.
model_relative_errors3 = np.abs(model_responses2 - responses) / np.abs(
    responses
)

# Visualize the results.
utils.plot_comparison(
    frequencies,
    responses,
    model_responses3,
    model_relative_errors3,
)
plt.show()

## Summary: The Complete and Practical Loewner Framework

With all the components together, it is finally time to apply the truncated Loewner framework with realification to a large data set. As example, we increase the size of the mass-spring-damper system to a chain with $k = 1000$ masses.

In [ ]:
large_model = utils.MassSpringDamper(
    num_masses = 1000, mass = 1.0, damping = 0.1, stiffness = 10.0
)

We observe data for $300$ frequencies.

In [ ]:
num_samples = 300

training_frequencies = np.logspace(-2, 2, num_samples)
training_responses   = large_model.transfer_function(1j * training_frequencies)

Now, we repeat the same steps from before, combining the two extensions:

1. Split the data and add the complex conjugates to allow for realification.

2. Construct the Loewner matrices.

3. Transform the Loewner matrices from complex to real entries.

4. Determine a suitable truncation rank.

5. Truncate the Loewner system to a real-valued low-dimensional model.

In [ ]:
# Split into left and right data sets AND add in the conjugates.
mu_l = insert_conjugates(1j * training_frequencies[::2])
mu_r = insert_conjugates(1j * training_frequencies[1::2])
g_l  = insert_conjugates(training_responses[::2])
g_r  = insert_conjugates(training_responses[1::2])

# Construct the transformation matrix J.
J = sparse.kron(
    sparse.eye(num_samples // 2),
    sparse.coo_matrix((1 / np.sqrt(2)) * np.array([[1, 1j], [1, -1j]])),
)

# Construct Loewner matrices.
L, Ls, BL, CL = loewner_matrices(mu_l, g_l, mu_r, g_r)

# Transform the Loewner system to real coordinates.
AL = -J.conj().T @ (Ls @ J)
bL = J.conj().T @ g_l
cL = g_r @ J
EL = -J.conj().T @ (L @ J)

# Compute the singular value decompositions of the pencil matrices.
Phi_1, s1, Psi_1T = la.svd(np.vstack((EL, AL)))  # [-L; -Ls]
Phi_2, s2, Psi_2T = la.svd(np.hstack((EL, AL)))  # [-L, -Ls]

# Visualization singular value decay to determine rank.
utils.plot_singular_values(s1, s2)
plt.show()

The given data has a very steep singular value decay, so we can expect that a low-dimensional model will be sufficient to approximate the data well. Here, we use a relative tolerance of $10^{-6}$ on the singular values of the Loewner pencil to select the reduced dimension $r$. Experiment with other tolerances and ranks to see how the approximation quality changes.

In [ ]:
# Determine a truncation rank.
tol = 1e-06
r   = np.count_nonzero(s1 > s1[0] * tol)
print(f"Reduced dimension: {r}")

# Compute the reduced-order system.
Ar = Phi_2[:, :r].T @ (AL.real @ Psi_1T[:r, :].T)
br = Phi_2[:, :r].T @ bL.real
cr = cL.real @ Psi_1T[:r, :].T
Er = Phi_2[:, :r].T @ (EL.real @ Psi_1T[:r, :].T)

# Define the ROM.
loewner_rom = utils.SISO(A = Ar, b = br, c = cr, E = Er)

Finally, we check the quality of our approximation with respect to the given data. Since in most applications, only data is given rather than the actual system, this is the more practical approach for checking performance. In this sense, it is not uncommon to go back and choose a larger rank if the approximation quality appears to be insufficient.

In [ ]:
# Frequency response of the approximation in the given data points.
rom_responses = loewner_rom.transfer_function(1j * training_frequencies)

rom_relative_error = np.abs(rom_responses - training_responses) / np.abs(
    training_responses
)

axes = utils.plot_comparison(
    training_frequencies, training_responses, rom_responses, rom_relative_error
)
axes[0].legend(loc="lower left")
plt.show()

This concludes the tutorial example.